# dynamic_relax Calculation Demonstration

- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2017-07-24

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The dynamic_relax calculation dynamically relaxes an atomic configuration for a sepcified number of timesteps. Upon completion, the mean, $<X>$, and standard deviation, $\sigma_X$, of all thermo properties, $X$, are computed for a specified range of times. This method is meant to measure equilibrium properties of bulk materials, both at zero K and at various temperatures. 

__Disclaimer #1__: The calculation reports the standard deviation, $\sigma_X$ of the measured properties not the standard error of the mean, $\sigma_{<X>}$. The two are related to each other according to

$$ \sigma_{<X>} = \sigma_X \sqrt{\frac{C}{N}} $$,

where $N$ is the number of samples taken of $X$, and $C$ is a statistical inefficiency due to the autocorrelation of the measurements with time. Obtaining a proper estimate of $\sigma_{<X>}$ requires either estimating $C$ from the raw thermo data (not done here), or only taking measurements sporatically to ensure the samples are independent. 

__Disclaimer #2__: Good (low error) results requires running large simulations for a long time. The reasons for this are:

- Systems have to be large enough to avoid issues with fluctuations across the periodic boundaries.

- Runs must first let the systems equilibriate before meaningful measurements can be taken. 

- The standard deviation, $\sigma$, of thermo properties is proportional to the number of atoms, $N_a$ as $\sigma \propto \frac{1}{\sqrt{N_a}}$.

- The standard error, $\sigma_x$ of thermo properties is proportional to the number of samples taken, $N$ as $\sigma_x \propto \frac{1}{\sqrt{N}}$.
- - -

## Method and Theory

An initial system (and corresponding unit cell system) is supplied with box dimensions, $a_i^0$, close to the equilibrium values. A LAMMPS simulation then integrates the atomic positions and velocities for a specified number of timesteps. 

The calculation script allows for the use of different integration methods:

- nve integrates atomic positions without changing box dimensions or the system's total energy.

- npt integrates atomic positions and applies Nose-Hoover style thermostat and barostat (equilibriate to specified T and P).

- nvt integrates atomic positions and applies Nose-Hoover style thermostat (equilibriate to specified T).

- nph integrates atomic positions and applies Nose-Hoover style barostat (equilibriate to specified P).

- nve+l integrates atomic positions and applies Langevin style thermostat (equilibriate to specified T).

- nph+l integrates atomic positions and applies Nose-Hoover style barostat and Langevin style thermostat (equilibriate to specified T and P).

__Notes__ on the different control schemes:

- The Nose-Hoover barostat works by rescaling the box dimensions according to the measured system pressures.

- The Nose-Hoover thermostat works by rescaling the atomic velocities according to the measured system temperature (kinetic energy). Cannot be used with a temperature of 0 K.

- The Langevin thermostat works by modifying the forces on all atoms with both a dampener and a random temperature dependent fluctuation. Used at 0 K, only the force dampener is applied.

__Notes__ on run parameter values. The proper time to reach equilibrium (equilsteps), and sample frequency to ensure uncorrelated measurements (thermosteps) is simulation dependent. They can be influenced by the potential, timestep size, crystal structure, integration method, presence of defects, etc. The default values of equilsteps = 20,000 and thermosteps = 100 are based on general rule-of-thumb estimates for bulk crystals and EAM potentials, and may or may not be adequate.  

- - -

## Demonstration

### 1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import random
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

### 2. Assign values for the calculation's run parameters

#### 2.0 Set the calculation's working directory

In [2]:
calc_name = 'dynamic_relax'

# Check current working directory
cwd_name = os.path.basename(os.getcwd())

# Change working directory if needed
if cwd_name != calc_name:
    if not os.path.isdir(calc_name):
        os.mkdir(calc_name)
    os.chdir(calc_name)

#### 2.1 Specify system-specific paths

- __lammps_command__ is the LAMMPS command to use (required).

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

- __lib_directory__ defines the relative path to the iprPy library. This makes it easier to define paths to reference records later.

In [3]:
lammps_command = 'lmp_mpi'
mpi_command = 'mpiexec -localonly 4'
lib_directory = '../../../library'

#### 2.2 Specify the prototype system

- __system__ is an atomman.System to perform the scan on (required). 

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to use for ucell. 

- __prototype_path__ gives the path to the crystal_prototype reference record to use.

- __box_parameters__ defines the initial guess box parameters to scale ucell and system by.

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

In [4]:
prototype_name = 'A1--Cu--fcc'
box_parameters = [3.5, 3.5, 3.5]
sizemults = [10,10,10]

# Define prototype_path using lib_directory and prototype_name
prototype_path = os.path.abspath(os.path.join(lib_directory, 'crystal_prototype', prototype_name+'.json'))

# Create ucell by loading prototype record
ucell = am.load('system_model', prototype_path)[0]
print('# of atoms in ucell =', ucell.natoms)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

# Generate system by supersizing ucell
system = am.supersize(ucell, *sizemults)
print('# of atoms in system =', system.natoms)

# of atoms in ucell = 4
# of atoms in system = 4000


#### 2.3 Specify the potenital and elemental symbols

- __potential__ is the atomman.lammps.Potential representation of a LAMMPS implemented potential to use (required).

- __symbols__ is a list of the elemental model symbols of potential to associate with the unique atom types of system (required).

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for potential. 

- __potential_path__ gives the path to the potential_LAMMPS reference record to use.

- __potential_dir_path__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file).

In [5]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
symbols = ['Ni']

# Define potential_path and potential_dir_path using lib_directory and potential_name
potential_dir_path = os.path.abspath(os.path.join(lib_directory, 'potential_LAMMPS', potential_name))
potential_path = potential_dir_path + '.json'

# Create potential by loading LAMMPS-potential record
potential = lmp.Potential(potential_path, potential_dir_path)
print('Successfully loaded potential', potential)

Successfully loaded potential 1999--Mishin-Y--Ni--LAMMPS--ipr1


#### 2.4 Specify calculation-specific run parameters

- __integrator__ specifies the integrator style to use. Default value is 'nph+l' for temperature = 0, and 'npt' otherwise.

- __pressure_xx__ gives the xx component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_yy__ gives the yy component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_zz__ gives the zz component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __temperature__ gives the temperature to equilibriate the system to (nvt, npt, nve+l, and nph+l styles).

- __runsteps__ is the total number of integration timesteps to perform. Default value is 220000.
  
- __thermosteps__ specifies to output thermo values every this many timesteps. Default value is 100.
    
- __dumpsteps__ specifies to output dump files every this many timesteps. Default value is runsteps (only first and last steps are outputted as dump files).
    
- __equilsteps__ is the number of timesteps to equilibriate the system for. Only thermo values associated with timesteps greater than equilsteps will be included in the mean and standard deviation calculations. Default value is 20000. 

- __randomseed__ specifies a random number seed used to generate the initial atomic velocities and the Langevin thermostat fluctuations. Default value generates a new random integer every time.

In [6]:
integrator = 'npt'
pressure_xx = uc.set_in_units(0.0, 'GPa')
pressure_yy = uc.set_in_units(0.0, 'GPa')
pressure_zz = uc.set_in_units(0.0, 'GPa')
temperature = 300.0
runsteps = 220000
thermosteps = 100
dumpsteps = runsteps
equilsteps = 20000
randomseed = None

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 full_relax.template

This is the template for the LAMMPS script to perform the integration relaxation.

In [7]:
with open('full_relax.template', 'w') as f:
    f.write("""
#LAMMPS input script that performs a simple dynamic integration

<atomman_system_info>

<atomman_pair_info>

compute pe all pe/atom
compute ke all ke/atom
compute stress all stress/atom <stressterm>

thermo <thermosteps>
thermo_style custom step temp pe ke etotal lx ly lz pxx pyy pzz pyz pxz pxy
thermo_modify format float %.13e
timestep 0.001

<integrator_info>

dump dump_chkpt all custom <dumpsteps> *.dump id type xu yu zu c_pe c_ke & 
c_stress[1] c_stress[2] c_stress[3] c_stress[4] c_stress[5] c_stress[6]

run <runsteps> upto
""")

#### 3.2 integrator_info()

The integrator_info function generates the necessary LAMMPS command lines associated with the particular temperature/pressure integration scheme.

Arguments:

- __integrator__ specifies the integrator style to use. Default value is 'nph+l' for temperature = 0, and 'npt' otherwise.

- __p_xx__ gives the xx component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __p_yy__ gives the yy component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __p_zz__ gives the zz component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __temperature__ gives the temperature to equilibriate the system to (nvt, npt, nve+l, and nph+l styles).

- __randomseed__ specifies a random number seed used to generate the initial atomic velocities and the Langevin thermostat fluctuations. Default value generates a new random integer every time.

- __units__ is the LAMMPS units style to use for the simulation.

In [8]:
def integrator_info(integrator=None, p_xx=0.0, p_yy=0.0, p_zz=0.0, 
                    temperature=0.0, randomseed=None, units='metal'):
    """
    Generates LAMMPS commands for velocity creation and fix integrators. 
    
    Parameters
    ----------
    integrator : str or None, optional
        The integration method to use. Options are 'npt', 'nvt', 'nph',
        'nve', 'nve+l', 'nph+l'. The +l options use Langevin thermostat.
        (Default is None, which will use 'nph+l' for temperature == 0, and
        'npt' otherwise.)
    p_xx : float, optional
        The value to relax the x tensile pressure component to (default is
        0.0).
    p_yy : float, optional
        The value to relax the y tensile pressure component to (default is
        0.0).
    p_zz : float, optional
        The value to relax the z tensile pressure component to (default is
        0.0).
    temperature : float, optional
        The temperature to relax at (default is 0.0).
    randomseed : int or None, optional
        Random number seed used by LAMMPS in creating velocities and with
        the Langevin thermostat.  (Default is None which will select a
        random int between 1 and 900000000.)
    units : str, optional
        The LAMMPS units style to use (default is 'metal').
    
    Returns
    -------
    str
        The generated LAMMPS input lines for velocity create and fix
        integration commands.
    """
    
    # Get lammps units
    lammps_units = lmp.style.unit(units)
    Px = uc.get_in_units(p_xx, lammps_units['pressure'])
    Py = uc.get_in_units(p_yy, lammps_units['pressure'])
    Pz = uc.get_in_units(p_zz, lammps_units['pressure'])
    T = temperature
    
    # Check temperature and set default integrator
    if temperature == 0.0:
        if integrator is None: integrator = 'nph+l'
        assert integrator not in ['npt', 'nvt'], 'npt and nvt cannot run at 0 K'
    elif temperature > 0:
        if integrator is None: integrator = 'npt'
    else:
        raise ValueError('Temperature must be positive')
    
    # Set default randomseed
    if randomseed is None: randomseed = random.randint(1, 900000000)
    
    if   integrator == 'npt':
        start_temp = T*2.+1
        Tdamp = 100 * lmp.style.timestep(units)
        Pdamp = 1000 * lmp.style.timestep(units)
        int_info = '\n'.join([
                'velocity all create %f %i' % (start_temp, randomseed),
                'fix npt all npt temp %f %f %f &' % (T, T, Tdamp),
                '                x %f %f %f &' % (Px, Px, Pdamp),
                '                y %f %f %f &' % (Py, Py, Pdamp),
                '                z %f %f %f' % (Pz, Pz, Pdamp),
                ])
    
    elif integrator == 'nvt':
        start_temp = T*2.+1
        Tdamp = 100 * lmp.style.timestep(units)
        int_info = '\n'.join([
                'velocity all create %f %i' % (start_temp, randomseed),
                'fix nvt all nvt temp %f %f %f' % (T, T, Tdamp),
                ])
    
    elif integrator == 'nph':
        Pdamp = 1000 * lmp.style.timestep(units)
        int_info = '\n'.join([
                'fix nph all nph x %f %f %f &' % (Px, Px, Pdamp),
                '                y %f %f %f &' % (Py, Py, Pdamp),
                '                z %f %f %f' % (Pz, Pz, Pdamp),
                ])
    
    elif integrator == 'nve':
        int_info = 'fix nve all nve'
        
    elif integrator == 'nve+l':
        start_temp = T*2.+1
        Tdamp = 100 * lmp.style.timestep(units)
        int_info = '\n'.join([
                'velocity all create %f %i' % (start_temp, randomseed),
                'fix nve all nve',
                'fix langevin all langevin %f %f %f %i' % (T, T, Tdamp,
                                                           randomseed),
                ])

    elif integrator == 'nph+l':
        start_temp = T*2.+1
        Tdamp = 100 * lmp.style.timestep(units)
        Pdamp = 1000 * lmp.style.timestep(units)
        int_info = '\n'.join([
                'fix nph all nph x %f %f %f &' % (Px, Px, Pdamp),
                '                y %f %f %f &' % (Py, Py, Pdamp),
                '                z %f %f %f' % (Pz, Pz, Pdamp),
                'fix langevin all langevin %f %f %f %i' % (T, T, Tdamp,
                                                           randomseed),
                ])
    else:
        raise ValueError('Invalid integrator style')
    
    return int_info

#### 3.3 full_relax()

The full_relax function performs the LAMMPS simulation and extracts and averages the thermo properties.

- __lammps_command__ is the LAMMPS command to use.

- __system__ is an atomman.System to perform the scan on.

- __potential__ is the atomman.lammps.Potential representation of a LAMMPS implemented potential to use.

- __symbols__ is a list of element-model symbols for the Potential that correspond to system's atypes.
    
- __mpi_command__ MPI command for running LAMMPS in parallel. Default value is None (serial run).  

- __ucell__ is an atomman.System representing a fundamental unit cell of the system. If not given, ucell will be taken as system.

- __integrator__ specifies the integrator style to use. Default value is 'nph+l' for temperature = 0, and 'npt' otherwise.

- __pressure_xx__ gives the xx component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_yy__ gives the yy component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_zz__ gives the zz component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __temperature__ gives the temperature to equilibriate the system to (nvt, npt, nve+l, and nph+l styles).

- __runsteps__ is the total number of integration timesteps to perform. Default value is 220000.
  
- __thermosteps__ specifies to output thermo values every this many timesteps. Default value is 100.
    
- __dumpsteps__ specifies to output dump files every this many timesteps. Default value is runsteps (only first and last steps are outputted as dump files).
    
- __equilsteps__ is the number of timesteps to equilibriate the system for. Only thermo values associated with timesteps greater than equilsteps will be included in the mean and standard deviation calculations. Default value is 20000. 

- __randomseed__ specifies a random number seed used to generate the initial atomic velocities and the Langevin thermostat fluctuations. Default value generates a new random integer every time.

In [9]:
def full_relax(lammps_command, system, potential, symbols,
               mpi_command=None, ucell=None, p_xx=0.0, p_yy=0.0, p_zz=0.0,
               temperature=0.0, integrator=None, runsteps=220000,
               thermosteps=100, dumpsteps=None, equilsteps=20000,
               randomseed=None):
    """
    Performs a full dynamic relax on a given system at the given temperature
    to the specified pressure state.
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    system : atomman.System
        The system to perform the calculation on.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    symbols : list of str
        The list of element-model symbols for the Potential that correspond to
        system's atypes.
    mpi_command : str, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    ucell : atomman.System, optional
        The fundamental unit cell correspodning to system.  This is used to
        convert system dimensions to cell dimensions. If not given, ucell will
        be taken as system.
    p_xx : float, optional
        The value to relax the x tensile pressure component to (default is
        0.0).
    p_yy : float, optional
        The value to relax the y tensile pressure component to (default is
        0.0).
    p_zz : float, optional
        The value to relax the z tensile pressure component to (default is
        0.0).
    temperature : float, optional
        The temperature to relax at (default is 0.0).
    runsteps : int, optional
        The number of integration steps to perform (default is 220000).
    integrator : str or None, optional
        The integration method to use. Options are 'npt', 'nvt', 'nph',
        'nve', 'nve+l', 'nph+l'. The +l options use Langevin thermostat.
        (Default is None, which will use 'nph+l' for temperature == 0, and
        'npt' otherwise.)
    thermosteps : int, optional
        Thermo values will be reported every this many steps (default is
        100).
    dumpsteps : int or None, optional
        Dump files will be saved every this many steps (default is None,
        which sets dumpsteps equal to runsteps).
    equilsteps : int, optional
        The number of timesteps at the beginning of the simulation to
        exclude when computing average values (default is 20000).
    randomseed : int or None, optional
        Random number seed used by LAMMPS in creating velocities and with
        the Langevin thermostat.  (Default is None which will select a
        random int between 1 and 900000000.)
    
    Returns
    -------
    dict
        Dictionary of results consisting of keys:
        
        - **'a_lat'** (*float*) - The mean measured a lattice constant.
        - **'b_lat'** (*float*) - The mean measured b lattice constant.
        - **'c_lat'** (*float*) - The mean measured c lattice constant.
        - **'alpha_lat'** (*float*) - The alpha lattice angle.
        - **'beta_lat'** (*float*) - The beta lattice angle.
        - **'gamma_lat'** (*float*) - The gamma lattice angle.
        - **'E_coh'** (*float*) - The mean measured cohesive energy.
        - **'stress'** (*numpy.array*) - The mean measured stress state.
        - **'temp'** (*float*) - The mean measured temperature.
        - **'a_lat_std'** (*float*) - The standard deviation in the measured
          a lattice constant values.
        - **'b_lat_std'** (*float*) - The standard deviation in the measured
          b lattice constant values.
        - **'c_lat_std'** (*float*) - The standard deviation in the measured
          c lattice constant values.
        - **'E_coh_std'** (*float*) - The standard deviation in the measured
          cohesive energy values.
        - **'stress_std'** (*numpy.array*) - The standard deviation in the
          measured stress state values.
        - **'temp_std'** (*float*) - The standard deviation in the measured
          temperature values.
    """
    
    # Set ucell = system if ucell not given
    if ucell is None:
        ucell = system
    
    # Get ratios of lx, ly, and lz of system relative to a of ucell
    lx_a = system.box.a / ucell.box.a
    ly_b = system.box.b / ucell.box.b
    lz_c = system.box.c / ucell.box.c
    alpha = system.box.alpha
    beta =  system.box.beta
    gamma = system.box.gamma
    
    # Get lammps units
    lammps_units = lmp.style.unit(potential.units)
    
    #Get lammps version date
    lammps_date = iprPy.tools.check_lammps_version(lammps_command)['lammps_date']
    
    # Handle default values
    if dumpsteps is None:
        dumpsteps = runsteps
    
    # Define lammps variables
    lammps_variables = {}
    system_info = lmp.atom_data.dump(system, 'init.dat',
                                     units=potential.units,
                                     atom_style=potential.atom_style)
    lammps_variables['atomman_system_info'] = system_info
    lammps_variables['atomman_pair_info'] = potential.pair_info(symbols)
    integ_info = integrator_info(integrator=integrator,
                                 p_xx=p_xx, p_yy=p_yy, p_zz=p_zz,
                                 temperature=temperature,
                                 randomseed=randomseed,
                                 units=potential.units)
    lammps_variables['integrator_info'] = integ_info
    lammps_variables['thermosteps'] = thermosteps
    lammps_variables['runsteps'] = runsteps
    lammps_variables['dumpsteps'] = dumpsteps
    
    # Set compute stress/atom based on LAMMPS version
    if lammps_date < datetime.date(2014, 2, 12):
        lammps_variables['stressterm'] = ''
    else:
        lammps_variables['stressterm'] = 'NULL'
    
    # Write lammps input script
    template_file = 'full_relax.template'
    lammps_script = 'full_relax.in'
    with open(template_file) as f:
        template = f.read()
    with open(lammps_script, 'w') as f:
        f.write(iprPy.tools.filltemplate(template, lammps_variables,
                                         '<', '>'))
    
    # Run lammps 
    output = lmp.run(lammps_command, lammps_script, mpi_command)
    
    # Extract LAMMPS thermo data. 
    results = {}
    thermo = output.simulations[0]['thermo']
    
    # Load relaxed system from dump file
    last_dump_file = str(thermo.Step.values[-1])+'.dump'
    results['system_relaxed'] = lmp.atom_dump.load(last_dump_file)
    
    # Only consider values where Step >= equilsteps
    thermo = thermo[thermo.Step >= equilsteps]
    results['nsamples'] = len(thermo)
    
    # Get cohesive energy estimates
    natoms = results['system_relaxed'].natoms
    results['E_coh'] = uc.set_in_units(thermo.PotEng.mean() / natoms,
                                       lammps_units['energy'])
    results['E_coh_std'] = uc.set_in_units(thermo.PotEng.std() / natoms,
                                           lammps_units['energy'])
    
    # Get lattice constant estimates
    results['a_lat'] = uc.set_in_units(thermo.Lx.mean() / lx_a,
                                      lammps_units['length'])
    results['b_lat'] = uc.set_in_units(thermo.Ly.mean() / ly_b,
                                      lammps_units['length'])
    results['c_lat'] = uc.set_in_units(thermo.Lz.mean() / lz_c,
                                       lammps_units['length'])
    results['a_lat_std'] = uc.set_in_units(thermo.Lx.std() / lx_a,
                                           lammps_units['length'])
    results['b_lat_std'] = uc.set_in_units(thermo.Ly.std() / ly_b,
                                           lammps_units['length'])
    results['c_lat_std'] = uc.set_in_units(thermo.Lz.std() / lz_c,
                                           lammps_units['length'])
    results['alpha_lat'] = alpha
    results['beta_lat'] =  beta
    results['gamma_lat'] = gamma
    
    # Get system stress estimates
    pxx = uc.set_in_units(thermo.Pxx.mean(), lammps_units['pressure'])
    pyy = uc.set_in_units(thermo.Pyy.mean(), lammps_units['pressure'])
    pzz = uc.set_in_units(thermo.Pzz.mean(), lammps_units['pressure'])
    pxy = uc.set_in_units(thermo.Pxy.mean(), lammps_units['pressure'])
    pxz = uc.set_in_units(thermo.Pxz.mean(), lammps_units['pressure'])
    pyz = uc.set_in_units(thermo.Pyz.mean(), lammps_units['pressure'])
    results['stress'] = -1 * np.array([[pxx, pxy, pxz],
                                       [pxy, pyy, pyz],
                                       [pxz, pyz, pzz]])
    pxx = uc.set_in_units(thermo.Pxx.std(), lammps_units['pressure'])
    pyy = uc.set_in_units(thermo.Pyy.std(), lammps_units['pressure'])
    pzz = uc.set_in_units(thermo.Pzz.std(), lammps_units['pressure'])
    pxy = uc.set_in_units(thermo.Pxy.std(), lammps_units['pressure'])
    pxz = uc.set_in_units(thermo.Pxz.std(), lammps_units['pressure'])
    pyz = uc.set_in_units(thermo.Pyz.std(), lammps_units['pressure'])
    results['stress_std'] = np.array([[pxx, pxy, pxz],
                                      [pxy, pyy, pyz],
                                      [pxz, pyz, pzz]])

    # Get system temperature estimates
    results['temp'] = thermo.Temp.mean()
    results['temp_std'] = thermo.Temp.std()
    
    return results

### 4. Run calculation function(s)

In [10]:
results_dict = full_relax(lammps_command, 
                          system, 
                          potential,
                          symbols,
                          mpi_command = mpi_command,
                          ucell=ucell,
                          p_xx=pressure_xx,
                          p_yy=pressure_yy,
                          p_zz=pressure_zz,
                          temperature=temperature,
                          integrator=integrator,
                          runsteps=runsteps,
                          thermosteps=thermosteps,
                          dumpsteps=dumpsteps,
                          equilsteps=equilsteps,
                          randomseed=randomseed)

In [11]:
results_dict.keys()

['E_coh_std',
 'stress',
 'gamma_lat',
 'temp',
 'stress_std',
 'a_lat',
 'beta_lat',
 'b_lat',
 'E_coh',
 'c_lat',
 'alpha_lat',
 'c_lat_std',
 'temp_std',
 'nsamples',
 'a_lat_std',
 'system_relaxed',
 'b_lat_std']

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of length to display relaxed lattice constants in.
- __energy_unit__ is the unit of energy to display cohesive energies in.
- __pressure_unit__ is the unit of pressure to display Cij and relaxed stress state in.

In [12]:
length_unit = 'angstrom'
energy_unit = 'eV'
pressure_unit = 'GPa'

#### 5.2 Display number of samples and number of atoms

In [13]:
print('# of samples = ', results_dict['nsamples'])
print('# of atoms =   ', results_dict['system_relaxed'].natoms)

# of samples =  2001
# of atoms =    4000


#### 5.3 Print Ecoh and lattice constants of relaxed ucell

In [14]:
print('Ecoh =      %9.6f' % uc.get_in_units(results_dict['E_coh'], energy_unit), energy_unit)
print('std(Ecoh) = %9.6f' % uc.get_in_units(results_dict['E_coh_std'], energy_unit), energy_unit)
print('a =         %9.6f' % uc.get_in_units(results_dict['a_lat'], length_unit), length_unit)
print('std(a) =    %9.6f' % uc.get_in_units(results_dict['a_lat_std'], energy_unit), length_unit)
print('b =         %9.6f' % uc.get_in_units(results_dict['b_lat'], length_unit), length_unit)
print('std(b) =    %9.6f' % uc.get_in_units(results_dict['b_lat_std'], energy_unit), length_unit)
print('c =         %9.6f' % uc.get_in_units(results_dict['c_lat'], length_unit), length_unit)
print('std(c) =    %9.6f' % uc.get_in_units(results_dict['c_lat_std'], energy_unit), length_unit)
print('alpha =     %9.6f' % results_dict['alpha_lat'])
print('beta =      %9.6f' % results_dict['beta_lat'])
print('gamma =     %9.6f' % results_dict['gamma_lat'])

Ecoh =      -4.413040 eV
std(Ecoh) =  0.000484 eV
a =          3.532964 angstrom
std(a) =     0.001598 angstrom
b =          3.533034 angstrom
std(b) =     0.001686 angstrom
c =          3.533102 angstrom
std(c) =     0.001813 angstrom
alpha =     90.000000
beta =      90.000000
gamma =     90.000000


#### 5.4 Check stress state of relaxed system

In [15]:
print('stress ('+pressure_unit+') =')
for Si in uc.get_in_units(results_dict['stress'], pressure_unit):
    print('[%9.4f %9.4f %9.4f]' % tuple(Si))
print()
print('std(stress) ('+pressure_unit+') =')
for Si in uc.get_in_units(results_dict['stress_std'], pressure_unit):
    print('[%9.4f %9.4f %9.4f]' % tuple(Si))

stress (GPa) =
[   0.0007    0.0015    0.0006]
[   0.0015    0.0011   -0.0004]
[   0.0006   -0.0004    0.0007]

std(stress) (GPa) =
[   0.0879    0.0257    0.0245]
[   0.0257    0.0915    0.0252]
[   0.0245    0.0252    0.0932]


#### 5.5 Check temperature state of relaxed system

In [16]:
print('T =      %9.6f K' % results_dict['temp'])
print('std(T) = %9.6f K' % results_dict['temp_std'])

T =      300.176572 K
std(T) =  3.826057 K
